# Perf Tools Analisys

Neste notebook iremos verificar as analises referentes ao perf tools da implementação do loop fusion, verificando quanto de acesso ao cache misses e cache references é utilizado.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import time
from IPython.display import display
from collections import Counter
import os
import re
import math
import random
pd.set_option('display.max_rows', 300)
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
#loads the csv as a dataframe and standardizes the algorithm names 
def load_df(filename):
    df = pd.read_csv(filename, index_col=False)
    return select_columns_and_rename_values(df)

In [3]:
#filters by substring (there are multiple OzaBag algorithms)
def filter_by_substring_algorithm(df, string):
    aux = df[df['algorithm'].str.contains(string, regex=False)]
    ret = aux
    if string == 'OB':
        ret = aux[~aux.algorithm.str.contains("Adwin|ASHT")]
    elif string == 'OzaBag':
        ret = aux[(aux.algorithm.str.contains(string)) & (~aux.algorithm.str.contains("Adwin|ASHT"))]
    return ret

In [4]:
#standardize algorithm names
def select_columns_and_rename_values(df):
    df.algorithm = df.algorithm.str.replace("Executor", "")
    df['algorithm'] = df["algorithm"].str.replace("OzaBag", "OB")
    df['algorithm'] = df["algorithm"].str.replace("AdaptiveRandomForest", "ARF")
    df['algorithm'] = df["algorithm"].str.replace("SequentialChunk", "SeqMB")
    df['algorithm'] = df["algorithm"].str.replace("OB$", "OBSequential")
    df['algorithm'] = df['algorithm'].str.replace("LeveragingBag", "LBagSequential")
    df['algorithm'] = df['algorithm'].str.replace("Adwin$", "AdwinSequential")
    df['algorithm'] = df['algorithm'].str.replace("CHUNK", "MB")
    df['algorithm'] = df['algorithm'].str.replace("MAXChunk", "MB")
    df['algorithm'] = df['algorithm'].str.replace("StreamingRandomPatches", "SRP")
    df['algorithm'] = df['algorithm'].str.replace("SRP$", "SRPSequential")
    df['algorithm'] = df['algorithm'].str.replace("OBASHT$", "OBASHTSequential")
    df.batch_size.unique()
    return df

In [37]:
import re
MILLION = 1000000
def parse_perf_file(fname, methodology='MB'):
    spname = fname.split('/')[-1].split('-')
    
    cores = 4
    if methodology == 'Sequential':
        cores = 1

    lineDict = {
        'dataset': spname[1],
        'algorithm': spname[2],
        'ensemble': spname[3],
        'methodology': methodology,
        'cores': cores,
        'batch_size': spname[5],
        'rate': spname[6],
        'cache-misses': None,
        'cache-references': None
    }

    with open (fname) as file:
        cache_misses = "0"
        cache_references = "0"
        for line in file:
            line = re.findall(r'\S+', line)
            try:
                if line[1]:
                    if line[1] == "cache-misses:u":
                        cache_misses = line[0]
                        
                    if line[1] == "cache-references:u":
                        cache_references = line[0]
            except IndexError:
                pass
    
    lineDict['cache-misses'] = float(cache_misses.replace(',', '')) / MILLION
    lineDict['cache-references'] = float(cache_references.replace(',', '')) / MILLION
            
    return lineDict

In [40]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/analise-perf/without-loop-fusion/second"
wantedCSVfilename = "loop-fusion-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [42]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response_mb = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response_mb.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'MB')
            )

df_mini_batching = pd.DataFrame.from_dict(response_mb)
display(df_mini_batching.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
16,GMSC,AdaptiveRandomForestExecutorMAXChunk,25,MB,4,50,1,"2,012.97","72,141.55"
5,GMSC,LBagExecutorMAXChunk,25,MB,4,50,1,"1,454.33","55,740.63"
1,GMSC,OzaBagASHTExecutorMAXChunk,25,MB,4,50,1,595.58,"18,683.21"
10,GMSC,OzaBagAdwinExecutorMAXChunk,25,MB,4,50,1,551.87,"23,330.91"
19,GMSC,OzaBagExecutorMAXChunk,25,MB,4,50,1,649.13,"18,903.30"
15,GMSC,StreamingRandomPatchesExecutorMAXChunk,25,MB,4,50,1,"3,950.59","149,357.07"
18,airlines,AdaptiveRandomForestExecutorMAXChunk,25,MB,4,50,1,"45,014.25","2,023,916.79"
2,airlines,LBagExecutorMAXChunk,25,MB,4,50,1,"63,988.26","2,571,551.79"
17,airlines,OzaBagASHTExecutorMAXChunk,25,MB,4,50,1,"3,690.39","272,844.31"
9,airlines,OzaBagAdwinExecutorMAXChunk,25,MB,4,50,1,"18,586.31","820,109.51"


In [45]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/analise-perf/loop-fusion/second"
wantedCSVfilename = "loop-fusion-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [46]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'MB-LF')
            )

df = pd.DataFrame.from_dict(response)
display(df.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
16,GMSC,AdaptiveRandomForestExecutorMAXChunk,25,MB-LF,4,50,1,"1,874.37","70,374.48"
5,GMSC,LBagExecutorMAXChunk,25,MB-LF,4,50,1,"1,407.62","57,031.14"
1,GMSC,OzaBagASHTExecutorMAXChunk,25,MB-LF,4,50,1,361.69,"12,073.58"
10,GMSC,OzaBagAdwinExecutorMAXChunk,25,MB-LF,4,50,1,522.77,"23,365.59"
19,GMSC,OzaBagExecutorMAXChunk,25,MB-LF,4,50,1,425.65,"13,531.57"
15,GMSC,StreamingRandomPatchesExecutorMAXChunk,25,MB-LF,4,50,1,"3,565.47","140,518.68"
18,airlines,AdaptiveRandomForestExecutorMAXChunk,25,MB-LF,4,50,1,"44,823.64","1,992,470.50"
2,airlines,LBagExecutorMAXChunk,25,MB-LF,4,50,1,"64,793.89","2,545,632.94"
17,airlines,OzaBagASHTExecutorMAXChunk,25,MB-LF,4,50,1,"1,767.09","150,217.82"
9,airlines,OzaBagAdwinExecutorMAXChunk,25,MB-LF,4,50,1,"18,328.13","817,137.55"


In [49]:
#Folder inside results directory that contains all the MOA dump files for these experiments
%cd /Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/
folderMOADumps = "/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching/results/analise-perf/loop-fusion-sequential/second"
wantedCSVfilename = "sequential-perf"

/Users/reginaldoluisdeluna/Documents/Ufscar/comparison-xue3m-minibatching


In [50]:
resultsFolder = f"{folderMOADumps}"
csvFile = f"parsed_csvs/{wantedCSVfilename}"
directory = os.fsencode(resultsFolder)
header_printed = False

response = []
with open(f"{csvFile}", "w+") as output:
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.startswith("perf-"):
            response.append(
                parse_perf_file(f'{os.fsdecode(directory)}/{filename}', 'Sequential')
            )

df = pd.DataFrame.from_dict(response)
display(df.sort_values(by=['dataset', 'algorithm']).drop_duplicates())

,dataset,algorithm,ensemble,methodology,cores,batch_size,rate,cache-misses,cache-references
20,GMSC,AdaptiveRandomForestSequential,25,Sequential,1,1,1,"2,121.13","75,026.09"
6,GMSC,LeveragingBag,25,Sequential,1,1,1,"1,605.01","51,203.12"
7,GMSC,OzaBag,25,Sequential,1,1,1,707.52,"17,166.12"
13,GMSC,OzaBagASHT,25,Sequential,1,1,1,696.86,"17,899.88"
18,GMSC,OzaBagAdwin,25,Sequential,1,1,1,811.74,"21,499.92"
14,GMSC,StreamingRandomPatches,25,Sequential,1,1,1,"4,459.59","159,692.59"
10,airlines,AdaptiveRandomForestSequential,25,Sequential,1,1,1,"1,357.53","55,305.69"
17,airlines,LeveragingBag,25,Sequential,1,1,1,"1,683.33","67,367.87"
0,airlines,OzaBag,25,Sequential,1,1,1,619.27,"21,882.42"
2,airlines,OzaBagASHT,25,Sequential,1,1,1,611.63,"21,782.19"
